#### 두 개의 투자전략을 클래스로 선언
1. 생성자 함수
    - 클래스가 생성될 때 최초로 한번만 실행이 되는 함수
    - 객체 변수(self 변수)에 데이터프레임, 기준이 되는 컬럼명, 시작시간, 종료시간
2. BuyandHold 함수
    - 생성자 함수에서 만든 객체 변수를 이용하여 백태스팅
    - 결과와 누적 수익률을 리턴
3. Bollinger 함수
    - 밴드 생성 함수
        - 상단 밴드, 하단 밴드, 이동평균선 생성
    - 거래 내역 추가 함수
        - 밴드를 기준으로 거래 내역을 생성
    - 수익률 계산 함수
        - 매도 시 수익률 발생, 데이터프레임과 누적수익률 리턴


In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

C:\Users\User\AppData\Local\Temp\ipykernel_19980\3713710582.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [36]:
class invest:
    # 생성자 함수
    def __init__(self, _df, _col = 'Adj Close', _start = '2010-01-01', _end = datetime.now()):
        # _df의 결측치, 무한대를 제외시킨다.
        flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        self.df = _df.loc[~flag, ]
        # 데이터프레임에서 컬럼에 Date가 포함되어 있는가?
        if 'Date' in self.df.columns:
            self.df.set_index('Date', inplace=True)
        # 인덱스를 시계열데이터로 변경
        self.df.index = pd.to_datetime(self.df.index, format='%Y-%m-%d')
        self.df = self.df[[_col]]
        self.col = _col
        try:
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str':
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else:
                self.end = _end
        except: 
            print('투자기간의 인자값의 포멧이 잘못되었습니다.(YYYY-mm-dd)')
    ## buyandhold 함수를 생성
    def buyandhold(self):
        # 투자기간으로 데이터를 필터링
        result = self.df.loc[self.start : self.end, ]
        # 일일 수익률 컬럼을 생성
        result['rtn'] = (result[self.col].pct_change()+1).fillna(1)
        # 누적 수익률 컬럼을 생성
        result['acc_rtn'] = result['rtn'].cumprod() #누적 곱
        # 총 누적 수익률 데이터만 추출
        acc_rtn = result.iloc[-1, ]['acc_rtn']
        return result, acc_rtn
    # bollinger 함수
    # 밴드 생성 함수 만들기
    def create_band(self, _cnt = 20):
        result = self.df.copy()
        # 이동 평균선 생성
        result['center'] = result[self.col].rolling(_cnt).mean()
        # 상단 밴드를 생성
        result['ub'] = result['center'] + (2 * result[self.col].rolling(_cnt).std())
        # 하단 밴드를 
        result['lb'] = result['center'] - (2 * result[self.col].rolling(_cnt).std())
        # 투자기간으로 데이터를 필터
        result = result.loc[self.start : self.end, ]
        return result
        

In [37]:
AAPL = pd.read_csv('../../../csv/AAPL.csv')

In [38]:
class1 = invest(AAPL)

In [39]:
bnh_df, bnh_rtn = class1.buyandhold()

C:\Users\User\AppData\Local\Temp\ipykernel_19980\1586869026.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rtn'] = (result[self.col].pct_change()+1).fillna(1)
C:\Users\User\AppData\Local\Temp\ipykernel_19980\1586869026.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['acc_rtn'] = result['rtn'].cumprod() #누적 곱


In [40]:
bnh_rtn

7.436513727083075

In [41]:
bnh_df.tail()

,Adj Close,rtn,acc_rtn
Date,,,
2019-06-18,198.449997,1.023518,7.409631
2019-06-19,197.869995,0.997077,7.387975
2019-06-20,199.460007,1.008036,7.447342
2019-06-21,198.779999,0.996591,7.421952
2019-06-24,199.169998,1.001962,7.436514


In [42]:
class1.create_band()

,Adj Close,center,ub,lb
Date,,,,
2010-01-04,26.782711,25.037723,27.046734,23.028713
2010-01-05,26.829010,25.169503,27.288098,23.050908
2010-01-06,26.402260,25.307290,27.366449,23.248130
2010-01-07,26.353460,25.436879,27.410937,23.462821
2010-01-08,26.528664,25.525609,27.529742,23.521475
...,...,...,...,...
2019-06-18,198.449997,185.432500,201.032574,169.832427
2019-06-19,197.869995,185.996000,202.558154,169.433846
2019-06-20,199.460007,186.830000,204.361771,169.298229
